<a href="https://colab.research.google.com/github/jonkstro/Webscraping_Olx/blob/main/01_Webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Workshop de Previsões de valores de venda de automóveis com Webscraping e Machine Learning

Git: https://github.com/gabriellimagomes15/previsao_valor_automoveis

##IMPORTAR AS BIBLIOTECAS

In [2]:
cd /content/drive/MyDrive/Colab Notebooks/Workshop_Info_Dados/dados_olx


[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Workshop_Info_Dados/dados_olx'
/content


In [3]:
# importar os pacotes do webscraping

from bs4 import BeautifulSoup
import requests
import time
import random
import pandas as pd
from datetime import datetime

##CAPTURANDO A PÁGINA

In [4]:
# definindo a página que será extraída
url_inicial = 'https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios?o=1'

# acessando a pagina
pagina_inicial = requests.get(url_inicial)
print(pagina_inicial) # imprime a resposta http da página (200 = OK; 400 = Erro)

<Response [403]>


In [5]:
# definindo a página que será extraída:
url_inicial = 'https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios?o=1'

# criando USER-AGENTE para realizar autenticação no site
# sem isso o olx vai bloquear o robô
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36",
}

# acessando a página
pagina_inicial = requests.get(url_inicial, headers = headers)
print(pagina_inicial) # é pra imprimir Cod 200 (http:200 = ok)

<Response [200]>


In [6]:
# converter a página em bytes para um objeto do beautifulsoup (que irá ter todas as funções pra fazer o scrap)
pagina_inicial = BeautifulSoup(pagina_inicial.content, 'html.parser')
print(type(pagina_inicial))

<class 'bs4.BeautifulSoup'>


In [7]:
# imprimir a página toda bonitinha pelo beutifulsoup
#print(pagina_inicial.prettify())

##ACESSANDO E EXTRAINDO ELEMENTOS DA PÁGINA


---




### Acessando Elementos da Página
* a função **find_all** acessa todos os elementos encontrados com o valor informado, seja, class, tag name ou algum outro seletor.
* o retorno dela será uma lista
  * **negrito** deve ter atenção se outros elementos na páginas que não queremos, possuem a mesma identificação, no caso de class
  * no caso da página da OLX, cada página terá 50 metadados dos anúncios. Mas como já citado, existem anúncios que utilizam outras classes.


In [9]:
#div = class="sc-1fcmfeb-0 FBZzf"

# extraindo vários anuncios
anuncios = pagina_inicial.find_all(class_="sc-1fcmfeb-0 FBZzf")
anuncios[:5] # imprimir os 5 primeiros itens da lista

[]

In [10]:
# convertendo os 5 primeiros anúncios para texto
[i.get_text() for i in anuncios] [:5]

[]

In [11]:
# extraindo os 5 primeiros títulos dos anúncios
titulos = pagina_inicial.find_all(class_="kgl1mq-0 iYdPim sc-bdVaJa daxpJj")
titulos[:5]

[]

In [12]:
# extraindo textos dos titulos dos anuncios
titulos = [i.get_text() for i in titulos]
titulos [:5]

[]

### Extração completa dos anúncios
* Agora será feita a extração de todos os dados que se deseja.

In [ ]:
# extraindo id dos anuncios:
id = pagina_inicial.find_all(class_="sc-12rk7z2-1")
ids = [i.get_attribute_list("data-lurker_list_id")[0] for i in id]

# extraindo link dos anuncios:
tags_links = pagina_inicial.find_all(name = "a", class_ = "sc-12rk7z2-1")
links = [i.get_attribute_list("href")[0] for i in tags_links]

# extraindo titulo dos anuncios:
titulo = pagina_inicial.find_all(class_="kgl1mq-0")
titulo = [i.get_text() for i in titulo]

# extraindo infos dos anuncios:
infos = pagina_inicial.find_all(class_="sc-1ftm7qz-2")
infos = [i.get_text() for i in infos]

# extraindo valor dos anuncios:
valor = pagina_inicial.find_all(class_="m7nrfa-0")
valor = [i.get_text() for i in valor]

# extraindo local dos anuncios:
local = pagina_inicial.find_all(class_="sc-1c3ysll-1")
local = [i.get_text() for i in local]

# extraindo data dos anuncios:
data = pagina_inicial.find_all(class_="sc-11h4wdr-0")
data = [i.get_text() for i in data]


In [ ]:
# verificar a quantidade de itens por variável [maximo por pagina]
len(ids), len(links), len(titulo), len(infos), len(valor), len(local), len(data)

(50, 50, 50, 50, 50, 50, 50)

In [ ]:
# vizualizando anuncios completo nas variáveis
i = 49
ids[i], links[i], titulo[i], infos[i], valor[i], local[i], data[i]

('986278408',
 'https://rj.olx.com.br/serra-angra-dos-reis-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/saveiro-tl-mbvs-2018-1-6-gnv-986278408',
 ' SAVEIRO TL MBVS 2018 1.6 GNV',
 '52.600 km\xa0\xa0|\xa0\xa0Câmbio: Manual\xa0\xa0|\xa0\xa0Gás Natural\xa0\xa0|\xa0\xa0',
 'R$ 66.900',
 'Barra Mansa -  RJ',
 'Hoje, 09:16')

##ESTRUTURANDO DADOS EM UM DATAFRAME


In [ ]:
# criar dataframe pra armazenar os dados
df = pd.DataFrame( list(zip(ids, titulo, infos, valor, local, data, links)),
                  columns = ['ID', 'TITULO', 'INFOS', 'VALOR', 'LOCAL', 'DATA_ANUNCIO', 'LINK'] )
df.head()

,ID,TITULO,INFOS,VALOR,LOCAL,DATA_ANUNCIO,LINK
0,1041364469,Palio Fire 2014/2015 |,75.000 km |,R$ 25.000,Feira de Santana - BA,"Hoje, 09:17",https://ba.olx.com.br/regiao-de-feira-de-santa...
1,1041364453,Honda Civic 2010 Lxs Automático,109.000 km | Câmbio: Automático | Flex |,R$ 44.500,São José do Rio Preto - SP,"Hoje, 09:17",https://sp.olx.com.br/regiao-de-sao-jose-do-ri...
2,1041364444,Toyota Hilux 4x4 SW4 Gasolina 1997,200.000 km | Câmbio: Manual | Gasolina |,R$ 42.000,São Sebastião da Amoreira - PR,"Hoje, 09:17",https://pr.olx.com.br/regiao-de-londrina/autos...
3,979399212,Vendo kombi corujinha 1973,1 km | Câmbio: Manual | Gasolina |,R$ 17.500,Campo Grande - MS,"Hoje, 09:17",https://ms.olx.com.br/mato-grosso-do-sul/autos...
4,1041364397,Carro,97.000 km | Câmbio: Manual | Flex |,R$ 48.000,Campo Bom - RS,"Hoje, 09:17",https://rs.olx.com.br/regioes-de-porto-alegre-...


In [ ]:
# salvando os links no csv
datahora = datetime.now().strftime('%Y%m%d%H%M')
df.to_csv(f'metadados_olx_{datahora}.csv', index = False)

##AUTOMATIZANDO A COLETA DOS DADOS

* Até o momento foi feito a coleta de **UMA** página.
* Como temos várias páginas, é preciso automatizar essa tarefa para as demais páginas.

###Fazer o Web scraping geral

In [ ]:
%%time
## DATAFRAME FINAL COM OS DADOS
df_metadados = pd.DataFrame()

## DATA QUE A COLETA FOI REALIZADA. SERÁ UTILIZADA PARA TER UM CONTROLE DE TEMPO DOS DADOS 
data_coleta = datetime.now().date()

print(f"==== COLETA INICIADA ==== ")
for pagina in range(1,5):
  print(f"{pagina}",end=" - ")

  ## URL COM A PAGINAÇÃO (pagina)
  url_inicial = f"https://olx.com.br/autos-e-pecas/carros-vans-e-utilitarios?o={pagina}"
  pagina_inicial = requests.get(url_inicial, headers=headers)
  print(pagina_inicial) # response 200 conexão OK, outros valores, houve algum problema

  # CONVERTENDO PÁGINA BYTES PARA OBJETO BEAUTIFULSOUP(que irá ter todas as funções/métodos para fazer o scraping) 
  pagina_inicial = BeautifulSoup(pagina_inicial.content, 'html.parser')
  
  ## EXTRAINDO ID DO ANUNCIO
  id = pagina_inicial.find_all(class_="sc-12rk7z2-1")
  ids = [x.get_attribute_list("data-lurker_list_id")[0] for x in id]

  ## EXTRAINDO LINK ANUNCIO
  tags_links = pagina_inicial.find_all(name = "a", class_ = "sc-12rk7z2-1")
  links = [ x.get_attribute_list("href")[0] for x in tags_links]

  ## EXTRAINDO TÍTULO ANUNCIO
  titulo = pagina_inicial.find_all(class_="kgl1mq-0")
  titulo = [x.get_text() for x in titulo]

  ## EXTRAINDO INFOS ANUNCIO
  infos = pagina_inicial.find_all(class_="sc-1ftm7qz-2")
  infos = [x.get_text() for x in infos]

  ## EXTRAINDO VALOR ANUNCIO
  valor = pagina_inicial.find_all(class_="m7nrfa-0")
  valor = [x.get_text() for x in valor]

  ## EXTRAINDO LOCALIZAÇÃO ANUNCIO
  local = pagina_inicial.find_all(class_="sc-1c3ysll-1")
  local = [x.get_text() for x in local]

  ## EXTRAINDO DATA ANUNCIO
  data = pagina_inicial.find_all(class_="sc-11h4wdr-0")
  data = [x.get_text() for x in data]

  ## VERIFICANDO A QUANTIDADE DE ELEMENTOS PARA CADA VARIÁVEIS
  print(len(ids), len(links),len(titulo), len(infos), len(valor), len(local), len(data))

  ## CRIANDO DATAFRAME TEMPORÁRIO PARA ARMAZENADOS OS DADOS
  df = pd.DataFrame( list(zip(ids,titulo,infos,valor,local,data,links)),
                    columns = ["ID","TITULO","INFOS","VALOR","LOCAL","DATA_ANUNCIO","LINK"] )
  
  ## CONDIÇÃO PARA VERIFICAR SE DADOS FORAM COLETADOS
  if len(df) > 0:
    #inserindo no DF a data de coleta
    df.loc[:,'DATA_COLETA_METADADOS'] = data_coleta 

    ## CONCATENANDO DADOS COLETADOS NO DATAFRAME FINAL(df_metadado)
    df_metadados = pd.concat([df_metadados,df])

    print("salvando\n")
    dataHora = datetime.now().strftime('%Y%m%d%H%M')

    ## SALVANDO DF LINKS
    df_metadados.to_csv(f"metadados_olx_carros_{dataHora}.csv",index=False)

    ## TIME PARA GERAR TEMPO DE "ESPERA" ANTES DE REALIZAR OUTRA REQUISIÇÃO
    # PROCEDIMENTO REALIZADO PARA NÃO SOBRECARREGAR O SERVIDOR
    time.sleep(random.randint(1,5))

print(f"==== COLETA FINALIZADA ==== ")


==== COLETA INICIADA ==== 
1 - <Response [200]>
50 50 50 50 50 50 50
salvando

2 - <Response [200]>
50 50 50 50 50 50 50
salvando

3 - <Response [200]>
50 50 50 50 50 50 50
salvando

4 - <Response [200]>
50 50 50 50 50 50 50
salvando

==== COLETA FINALIZADA ==== 
CPU times: user 2.12 s, sys: 43.2 ms, total: 2.17 s
Wall time: 18.4 s


###Fazer o Web scraping por Estado do Brasil

In [ ]:
%%time
## DATAFRAME FINAL COM OS DADOS
df_metadados = pd.DataFrame()

## DATA QUE A COLETA FOI REALIZADA. SERÁ UTILIZADA PARA TER UM CONTROLE DE TEMPO DOS DADOS 
data_coleta = datetime.now().date()

print(f"==== COLETA INICIADA ==== ")

uf = ['ac', 'al', 'ap', 'am', 'ba', 'ce', 'df', 'es', 'go', 'ma', 'mt', 'ms', 'mg', 'pa',
      'pb', 'pr', 'pe', 'pi', 'rj', 'rn', 'rs', 'ro', 'rr', 'sc', 'sp', 'se', 'to']

for x in uf:
  print(f"Realizando coletas do UF {x}:")
  for pagina in range(1,5):
    print(f"{pagina}",end=" - ")

    ## URL COM A PAGINAÇÃO (pagina)
    url_inicial = f"https://{x}.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios?o={pagina}"
    pagina_inicial = requests.get(url_inicial, headers=headers)
    print(pagina_inicial) # response 200 conexão OK, outros valores, houve algum problema

    # CONVERTENDO PÁGINA BYTES PARA OBJETO BEAUTIFULSOUP(que irá ter todas as funções/métodos para fazer o scraping) 
    pagina_inicial = BeautifulSoup(pagina_inicial.content, 'html.parser')
    
    ## EXTRAINDO ID DO ANUNCIO
    id = pagina_inicial.find_all(class_="sc-12rk7z2-1")
    ids = [x.get_attribute_list("data-lurker_list_id")[0] for x in id]

    ## EXTRAINDO LINK ANUNCIO
    tags_links = pagina_inicial.find_all(name = "a", class_ = "sc-12rk7z2-1")
    links = [ x.get_attribute_list("href")[0] for x in tags_links]

    ## EXTRAINDO TÍTULO ANUNCIO
    titulo = pagina_inicial.find_all(class_="kgl1mq-0")
    titulo = [x.get_text() for x in titulo]

    ## EXTRAINDO INFOS ANUNCIO
    infos = pagina_inicial.find_all(class_="sc-1ftm7qz-2")
    infos = [x.get_text() for x in infos]

    ## EXTRAINDO VALOR ANUNCIO
    valor = pagina_inicial.find_all(class_="m7nrfa-0")
    valor = [x.get_text() for x in valor]

    ## EXTRAINDO LOCALIZAÇÃO ANUNCIO
    local = pagina_inicial.find_all(class_="sc-1c3ysll-1")
    local = [x.get_text() for x in local]

    ## EXTRAINDO DATA ANUNCIO
    data = pagina_inicial.find_all(class_="sc-11h4wdr-0")
    data = [x.get_text() for x in data]

    ## VERIFICANDO A QUANTIDADE DE ELEMENTOS PARA CADA VARIÁVEIS
    print(len(ids), len(links),len(titulo), len(infos), len(valor), len(local), len(data))

    ## CRIANDO DATAFRAME TEMPORÁRIO PARA ARMAZENADOS OS DADOS
    df = pd.DataFrame( list(zip(ids,titulo,infos,valor,local,data,links)),
                      columns = ["ID","TITULO","INFOS","VALOR","LOCAL","DATA_ANUNCIO","LINK"] )
    
    ## CONDIÇÃO PARA VERIFICAR SE DADOS FORAM COLETADOS
    if len(df) > 0:
      #inserindo no DF a data de coleta
      df.loc[:,'DATA_COLETA_METADADOS'] = data_coleta 

      ## CONCATENANDO DADOS COLETADOS NO DATAFRAME FINAL(df_metadado)
      df_metadados = pd.concat([df_metadados,df])

      print("salvando\n")
      dataHora = datetime.now().strftime('%Y%m%d%H%M')

      ## SALVANDO OS LINKS
      #df_metadados.to_csv(f"metadados_olx_carros_{dataHora}.csv",index=False)

      ## TIME PARA GERAR TEMPO DE "ESPERA" ANTES DE REALIZAR OUTRA REQUISIÇÃO
      # PROCEDIMENTO REALIZADO PARA NÃO SOBRECARREGAR O SERVIDOR
      time.sleep(random.randint(1,5))
  #SALVANDO OS LINKS
  df_metadados.to_csv(f"{x}_metadados.csv",index=False)
print(f"==== COLETA FINALIZADA ==== ")
